In [1]:
from lib import *

In [2]:
loan = pd.read_csv("./Loan_information.csv")

In [3]:
loan['OPNDT'] =  pd.to_datetime(loan['OPNDT'], format='%d/%m/%Y')
loan['END_OF_TENOR'] =  pd.to_datetime(loan['END_OF_TENOR'], format='%d/%m/%Y')
loan['LOAN_LENGTH'] = (loan['END_OF_TENOR'] - loan['OPNDT']).dt.days

In [4]:
loan.drop(['CLSDT', 'PHONE', 'ADDRESS', 'CREDIT_ACNO', 'Debit ACNO', 'DEALER_NAME', 'OPNDT', 'END_OF_TENOR'], axis=1, inplace=True)

In [5]:
loan_c = loan.drop(['LAST_REPAYMENT_DATE'], axis=1)

In [6]:
loan_c['LAST_REPAYMENT_AMOUNT'].fillna(value=loan_c['LAST_REPAYMENT_AMOUNT'].mean(), inplace=True)
loan_c.drop(['CUSTOMERID'], axis=1, inplace =True)


In [7]:
loan_c.NPL_STATUS.unique()

array(['Bad Debt', 'Doubtful', 'Sub-Standard', 'Standard', 'Watch'],
      dtype=object)

In [8]:
# create a new column based on the NPL_STATUS column that will be our target variable
loan_c['good_bad'] = np.where(loan_c.loc[:, 'NPL_STATUS'].isin(['Bad Debt', 'Doubtful']), 0, 1)
# Drop the original 'NPL_STATUS' column
loan_c.drop(columns = ['NPL_STATUS'], inplace = True)

In [9]:
corrMatrix = loan_c.corr()


/tmp/ipykernel_49181/91972575.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corrMatrix = loan_c.corr()


In [10]:
numerical_features = loan_c.select_dtypes(include='number')
object_features = loan_c.select_dtypes(include='object')
datetime_features = loan_c.select_dtypes(include='datetime')

In [11]:
loan_c.drop(datetime_features.columns, axis=1, inplace=True)

In [12]:
object_features.head()

,CUSTOMER_TYPE,CATNAME,CRD_PURPOSE,CRD_CLASSIFICATION,FIRSTDATE_OF_PRINCIPAL_PAYMENT,FIRSTDATE_OF_INTEREST_PAYMENT,BRNAME,ASDATE,NRC_NO,MIN_DUEDT
0,Single Customer,Hire Purchase (Upfront Method) Int 7%,Electronic Equipments,Agriculture/Livestocks,3/9/2018,3/9/2018,0001-Nay Pyi Taw,12/05/2021,05/KATHANA(N)109230,09/11/2018
1,Single Customer,Hire Purchase (Upfront Method) Int 7%,Electronic Equipments,Agriculture/Livestocks,3/9/2018,3/9/2018,0001-Nay Pyi Taw,12/05/2021,13/KATANA(N)125765,09/11/2018
2,Single Customer,Hire Purchase (Upfront Method) Int 7%,Electronic Equipments,Agriculture/Livestocks,3/16/2018,3/16/2018,0001-Nay Pyi Taw,12/05/2021,12/BHATAHTA(N)000138,16/11/2018
3,Single Customer,Hire Purchase (Upfront Method) Int 7%,Electronic Equipments,Agriculture/Livestocks,4/15/2018,4/15/2018,0001-Nay Pyi Taw,12/05/2021,09/PABHATHA(N)002545,15/11/2018
4,Single Customer,Hire Purchase (Upfront Method) Int 7%,Electronic Equipments,Agriculture/Livestocks,4/15/2018,4/15/2018,0001-Nay Pyi Taw,12/05/2021,12/MAYAKA(N)139588,15/11/2018


In [13]:
loan.CRD_CLASSIFICATION.unique()

array(['Agriculture/Livestocks'], dtype=object)

In [14]:
# dummy = pd.get_dummies(object_features['CUSTOMER_TYPE', 'CRD_PURPOSE', 'CRD_CLASSIFICATION', 'ACCOUNT_STATUS', 'MARTIAL_STATUS'], drop_first=True)

In [15]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
loan_c[['CUSTOMER_TYPE', 'CATNAME', 'CRD_PURPOSE']]= enc.fit_transform(loan_c[['CUSTOMER_TYPE', 'CATNAME', 'CRD_PURPOSE']])
# object_features['CRD_PURPOSE'] = enc.fit_transform(object_features['CRD_PURPOSE'])
# object_features['CRD_CLASSIFICATION'] = enc.fit_transform(object_features['CRD_CLASSIFICATION'])
# object_features['ACCOUNT_STATUS'] = enc.fit_transform(object_features['ACCOUNT_STATUS'])
# object_features['MARTIAL_STATUS'] = enc.fit_transform(object_features['MARTIAL_STATUS'])

In [16]:
loan_c.drop(['BRNAME', 'ASDATE',	'NRC_NO',	'MIN_DUEDT', 'CRD_CLASSIFICATION'], axis=1, inplace=True)

In [17]:
loan_c.head()

,CUSTOMER_TYPE,CATEGORYID,CATNAME,CRD_PURPOSE,CRLIMIT,DISBURSEMENT_AMT,NORMAL_PRINCIPAL_AMT,PRINCIPAL_AMT,FIRSTDATE_OF_PRINCIPAL_PAYMENT,PRINCIPAL_PAY_DAY,...,MONTH_DEBIT,MONTH_CREDIT,QUATER_DEBIT,QUATER_CREDIT,SEMI_ANNUAL_DEBIT,SEMI_ANNUAL_CREDIT,YEAR_DEBIT,YEAR_CREDIT,LOAN_LENGTH,good_bad
0,2.0,1067,26.0,3.0,413700.0,413700.0,137840.13,275859.87,3/9/2018,9,...,413700.0,275859.87,413700.0,275859.87,413700.0,275859.87,413700.0,275859.87,365,0
1,2.0,1067,26.0,3.0,595000.0,595000.0,198289.80,396710.20,3/9/2018,9,...,595000.0,396710.20,595000.0,396710.20,595000.0,396710.20,595000.0,396710.20,365,0
2,2.0,1067,26.0,3.0,560000.0,560000.0,186437.35,373562.65,3/16/2018,16,...,560000.0,373562.65,560000.0,373562.65,560000.0,373562.65,560000.0,373562.65,365,0
3,2.0,1067,26.0,3.0,559300.0,559300.0,202874.53,356425.47,4/15/2018,15,...,559300.0,356425.47,559300.0,356425.47,559300.0,356425.47,559300.0,356425.47,337,0
4,2.0,1067,26.0,3.0,707000.0,707000.0,256884.95,450115.05,4/15/2018,15,...,707000.0,450115.05,707000.0,450115.05,707000.0,450115.05,707000.0,450115.05,337,0


In [18]:
loan_c.drop(['FIRSTDATE_OF_PRINCIPAL_PAYMENT', 'FIRSTDATE_OF_INTEREST_PAYMENT'], axis=1, inplace = True)

In [19]:
columns = np.full((corrMatrix.shape[0],), True, dtype=bool)
for i in range(corrMatrix.shape[0]):
    for j in range(i+1, corrMatrix.shape[0]):
        if corrMatrix.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = numerical_features.columns[columns]

In [20]:
unselect = numerical_features.columns.difference(selected_columns)

In [21]:
#drop mutilcorr
loan_c.drop(unselect, axis=1, inplace = True)

In [22]:
import numpy as np
import matplotlib.pyplot as plt

MAX_VAL = 99999999999999
MIN_VAL = -MAX_VAL

def _bin_table(df, colname, n_bins = 10, qcut = None):
  X = df[[colname, 'good_bad']]
  X = X.sort_values(colname)
  coltype = X[colname].dtype

  if coltype in ['float', 'int']:
    if qcut is None:
      try:
        bins, thres = pd.qcut(X[colname], q = n_bins, retbins=True)
        # Thay thế threshold đầu và cuối của thres
        thres[0] = MIN_VAL
        thres[-1] = MAX_VAL
        bins, thres = pd.cut(X[colname], bins=thres, retbins=True)
        X['bins'] = bins
      except:
        print('n_bins must be lower to bin interval is valid!')
    else:
      bins, thres = pd.cut(X[colname], bins=qcut, retbins=True)
      X['bins'] = bins
  elif coltype == 'object':
    X['bins'] = X[colname]

  df_GB = pd.pivot_table(X, 
                index = ['bins'],
                values = ['good_bad'],
                columns = ['good_bad'],
                aggfunc = {
                    'good_bad':np.size
                })

  df_Count = pd.pivot_table(X, 
                index = ['bins'],
                values = ['good_bad'],
                aggfunc = {
                    'good_bad': np.size
                })
  
  if coltype in ['float', 'int']:
    df_Thres = pd.DataFrame({'Thres':thres[1:]}, index=df_GB.index)
  elif coltype == 'object':
    df_Thres = pd.DataFrame(index=df_GB.index)
    thres = None
  df_Count.columns = ['No_Obs']
  df_GB.columns = ['#BAD', '#GOOD']
  df_summary = df_Thres.join(df_Count).join(df_GB)
  return df_summary, thres

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

In [24]:
check = loan_c.drop(['good_bad'], axis=1)

In [25]:
check = check.drop(['CATNAME'], axis=1)

In [26]:
# function to calculate woe of categorical features

def woe_categorical(df, cat_feature, good_bad_df):
    df = pd.concat([df[cat_feature], good_bad_df], axis=1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                    df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df


# function to plot woe values 
# We set the default style of the graphs to the seaborn style. 
# sns.set()
# Below we define a function for plotting WoE across categories that takes 2 arguments: a dataframe and a number.
# def plot_by_woe(df_WoE, rotation_of_x_axis_labels = 0):
#     x = np.array(df_WoE.iloc[:, 0].apply(str))
#     y = df_WoE['WoE']
#     plt.figure(figsize=(18, 6))
#     plt.plot(x, y, marker = 'o', linestyle = '--', color = 'k')
#     plt.xlabel(df_WoE.columns[0])
#     plt.ylabel('Weight of Evidence')
#     plt.title(str('Weight of Evidence by ' + df_WoE.columns[0]))
#     plt.xticks(rotation = rotation_of_x_axis_labels)    


In [27]:
def _WOE(data, colname, n_bins = None, min_obs = 100, qcut = None):
  # Thống kê bins và lấy ra thres hold ban đầu
  df_summary, thres = _bin_table(data, colname, n_bins = n_bins, qcut = qcut)
  # Thay thế giá trị 0 của #BAD trong df_summary bằng 1 để không bị lỗi chia cho 0
  df_summary['#BAD'] = df_summary['#BAD'].replace({0:1})
  
  if qcut is not None:
    # Lọc bỏ threshold để tạo thành threshold mới mà thỏa mãn số lượng quan sát >= min_obs
    exclude_ind = np.where(df_summary['No_Obs'] <= min_obs)[0]
    if exclude_ind.shape[0] > 0:
      new_thres = np.delete(thres, exclude_ind)
      print('Auto combine {} bins into {} bins'.format(n_bins, new_thres.shape[0]-1))
      # Tính toán lại bảng summary
      df_summary, thres = _bin_table(data, colname, qcut=new_thres)
  
  new_thres = thres
  df_summary['GOOD/BAD'] = df_summary['#GOOD']/df_summary['#BAD']
  df_summary['%BAD'] = df_summary['#BAD']/df_summary['#BAD'].sum()
  df_summary['%GOOD'] = df_summary['#GOOD']/df_summary['#GOOD'].sum()
  df_summary['WOE'] = np.log(df_summary['%GOOD']/df_summary['%BAD'])
  df_summary['IV'] = (df_summary['%GOOD']-df_summary['%BAD'])*df_summary['WOE']
  df_summary['COLUMN'] = colname
  IV = df_summary['IV'].sum()
  print('Information Value of {} column: {}'.format(colname, IV))
  return df_summary, IV, new_thres

In [28]:
loan_d = loan_c[['CUSTOMER_TYPE', 'CRLIMIT', 'PRINCIPAL_AMT', 'PRINCIPAL_PAY_DAY',
       'INTEREST_PREPAID', 'NPL_DAYS', 'LAST_REPAYMENT_AMOUNT',
       'FEE_OUTSTANDING', 'VEXPRIREDATE', 'Down_payment%',
       'Down payment amount', 'LOAN_LENGTH', 'good_bad']]

In [29]:
loan_c.isna().sum()

CUSTOMER_TYPE            0
CATEGORYID               0
CATNAME                  0
CRD_PURPOSE              0
CRLIMIT                  0
PRINCIPAL_AMT            0
PRINCIPAL_PAY_DAY        0
DUE_AMT_OF_PRINCIPAL     0
INTEREST_PREPAID         0
NPL_DAYS                 0
MIN_NPL_DAYS             0
LAST_REPAYMENT_AMOUNT    0
TIMES                    0
FEE_OUTSTANDING          0
VEXPRIREDATE             0
Down_payment%            0
Down payment amount      0
LOAN_LENGTH              0
good_bad                 0
dtype: int64

In [30]:
WOE_dict=dict()
nbins = {'CUSTOMER_TYPE':1, 'CRLIMIT':33, 'PRINCIPAL_AMT':15, 'PRINCIPAL_PAY_DAY': 5, 'INTEREST_PREPAID':1, 'NPL_DAYS':2, 'VEXPRIREDATE':1,
 'LAST_REPAYMENT_AMOUNT':17, 'FEE_OUTSTANDING':2, 'Down_payment%':3, 'Down payment amount':3, 'LOAN_LENGTH':9}
for (col, bins) in nbins.items():
  df_summary, IV, thres = _WOE(loan_c, colname=col, n_bins=bins)
  WOE_dict[col] = {'table':df_summary, 'IV':IV}

Information Value of CUSTOMER_TYPE column: 0.0
Information Value of CRLIMIT column: 0.12580471004277544
Information Value of PRINCIPAL_AMT column: 0.049758510038969535
Information Value of PRINCIPAL_PAY_DAY column: 0.04977978125934047
Information Value of INTEREST_PREPAID column: 0.0
Information Value of NPL_DAYS column: 1.960933848540213
Information Value of VEXPRIREDATE column: 0.0
Information Value of LAST_REPAYMENT_AMOUNT column: 0.02984630213808569
Information Value of FEE_OUTSTANDING column: 2.025437516372508
Information Value of Down_payment% column: 0.028449448848436122
Information Value of Down payment amount column: 0.002212625574032738
Information Value of LOAN_LENGTH column: 0.106735646888015


In [31]:
columns = []
IVs = []
for col in loan_d.columns:
  if col != 'good_bad':
    columns.append(col)
    IVs.append(WOE_dict[col]['IV'])
df_WOE = pd.DataFrame({'column': columns, 'IV': IVs})

def _rank_IV(iv):
  if iv <= 0.02:
    return 'Useless'
  elif iv <= 0.1:
    return 'Weak'
  elif iv <= 0.3:
    return 'Medium'
  elif iv <= 0.5:
    return 'Strong'
  else:
    return 'suspicious'

df_WOE['rank']=df_WOE['IV'].apply(lambda x: _rank_IV(x))
df_WOE.sort_values('IV', ascending=False)

,column,IV,rank
7,FEE_OUTSTANDING,2.025438,suspicious
5,NPL_DAYS,1.960934,suspicious
1,CRLIMIT,0.125805,Medium
11,LOAN_LENGTH,0.106736,Medium
3,PRINCIPAL_PAY_DAY,0.049780,Weak
2,PRINCIPAL_AMT,0.049759,Weak
6,LAST_REPAYMENT_AMOUNT,0.029846,Weak
9,Down_payment%,0.028449,Weak
10,Down payment amount,0.002213,Useless
0,CUSTOMER_TYPE,0.000000,Useless


In [32]:
# entries_to_remove = ('Down payment amount', 'CUSTOMER_TYPE', 'ACCOUNT_STATUS')
entries_to_remove = ('TIMES')
for k in entries_to_remove:
    WOE_dict.pop(k, None)

In [33]:
for col in WOE_dict.keys():
  try:
    key = list(WOE_dict[col]['table']['WOE'].index)
    woe = list(WOE_dict[col]['table']['WOE'])
    d = dict(zip(key, woe))
    col_woe = col+'_WOE'
    loan_c[col_woe] = loan_c[col].map(d)
  except:
    print(col)

In [34]:
pd.set_option('display.max_columns', None)

In [35]:
loan_d[loan_d.isnull().any(axis=1)]

,CUSTOMER_TYPE,CRLIMIT,PRINCIPAL_AMT,PRINCIPAL_PAY_DAY,INTEREST_PREPAID,NPL_DAYS,LAST_REPAYMENT_AMOUNT,FEE_OUTSTANDING,VEXPRIREDATE,Down_payment%,Down payment amount,LOAN_LENGTH,good_bad


In [36]:
loan_WOE = loan_c.filter(like='_WOE', axis = 1)
loan_WOE['good_bad'] = loan_c['good_bad']

/tmp/ipykernel_49181/2287184674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_WOE['good_bad'] = loan_c['good_bad']


In [37]:
X = loan_WOE.filter(like='_WOE', axis = 1)
y = loan_WOE['good_bad']

In [38]:
from sklearn.model_selection import train_test_split

ids = np.arange(X.shape[0])
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split(X, y, ids, test_size = 0.2, stratify = y, shuffle = True, random_state = 42)


In [39]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
os = RandomOverSampler()
X_train, y_train = os.fit_resample(X_train, y_train)
y_train_series = pd.Series(y_train)

In [40]:
from sklearn.utils.extmath import weighted_mode
from sklearn.linear_model import LogisticRegression

logit_model = LogisticRegression(solver = 'lbfgs', max_iter=1000, fit_intercept=True, tol=0.0001, C=1, penalty='l2')
logit_model.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=1000)

In [41]:
def _KM(y_pred, n_bins):
  _, thresholds = pd.qcut(y_pred, q=n_bins, retbins=True)
  cmd_BAD = []
  cmd_GOOD = []
  BAD_id = set(np.where(y_test == 0)[0])
  GOOD_id = set(np.where(y_test == 1)[0])
  total_BAD = len(BAD_id)
  total_GOOD = len(GOOD_id)
  for thres in thresholds:
    pred_id = set(np.where(y_pred <= thres)[0])
    # Đếm % số lượng hồ sơ BAD có xác suất dự báo nhỏ hơn hoặc bằng thres
    per_BAD = len(pred_id.intersection(BAD_id))/total_BAD
    cmd_BAD.append(per_BAD)
    # Đếm % số lượng hồ sơ GOOD có xác suất dự báo nhỏ hơn hoặc bằng thres
    per_GOOD = len(pred_id.intersection(GOOD_id))/total_GOOD
    cmd_GOOD.append(per_GOOD)
  cmd_BAD = np.array(cmd_BAD)
  cmd_GOOD = np.array(cmd_GOOD)
  return cmd_BAD, cmd_GOOD, thresholds

In [42]:
import numpy as np

def _CreditScore(beta, alpha, woe, n = 12, odds = 1/4, pdo = -50, thres_score = 600):
  factor = pdo/np.log(2)
  offset = thres_score - factor*np.log(odds)
  score = (beta*woe+alpha/n)*factor+offset/n
  return score

_CreditScore(beta = 0.5, alpha = -1, woe = 0.15, n = 12)

42.2677896003704

In [43]:
logit_model.coef_[0]

array([ 0.        , -0.50171889,  0.8141791 ,  1.79432075,  0.        ,
        0.73068942,  0.        ,  1.39569791,  0.67177213,  1.01850624,
       -1.35703562, -0.17101453])

In [44]:
betas_dict = dict(zip(list(X_train.columns), logit_model.coef_[0]))
alpha = logit_model.intercept_[0]
betas_dict

{'CUSTOMER_TYPE_WOE': 0.0,
 'CRLIMIT_WOE': -0.5017188894137655,
 'PRINCIPAL_AMT_WOE': 0.8141791001006945,
 'PRINCIPAL_PAY_DAY_WOE': 1.794320754931938,
 'INTEREST_PREPAID_WOE': 0.0,
 'NPL_DAYS_WOE': 0.7306894153943922,
 'VEXPRIREDATE_WOE': 0.0,
 'LAST_REPAYMENT_AMOUNT_WOE': 1.3956979136188896,
 'FEE_OUTSTANDING_WOE': 0.6717721285194236,
 'Down_payment%_WOE': 1.0185062431806622,
 'Down payment amount_WOE': -1.3570356184270613,
 'LOAN_LENGTH_WOE': -0.17101453250337123}

In [45]:
columns = ['CUSTOMER_TYPE', 'CRLIMIT', 'PRINCIPAL_AMT', 'PRINCIPAL_PAY_DAY',
       'INTEREST_PREPAID', 'NPL_DAYS', 'LAST_REPAYMENT_AMOUNT',
       'FEE_OUTSTANDING', 'VEXPRIREDATE', 'Down_payment%',
       'Down payment amount', 'LOAN_LENGTH'
       ]  

In [46]:
cols = []
features = []
woes = []
betas = []
scores = []

for col in columns:
  for feature, woe in WOE_dict[col]['table']['WOE'].to_frame().iterrows():
    cols.append(col)
    # Add feature
    feature = str(feature)
    features.append(feature)    
    # Add woe
    woe = woe.values[0]
    woes.append(woe)
    # Add beta
    col_woe = col+'_WOE'
    beta = betas_dict[col_woe]
    betas.append(beta)
    # Add score
    score = _CreditScore(beta = beta, alpha = alpha, woe = woe, n = 12)
    scores.append(score)

df_WOE = pd.DataFrame({'Columns': cols, 'Features': features, 'WOE': woes, 'Betas':betas, 'Scores':scores})
df_WOE.head()

,Columns,Features,WOE,Betas,Scores
0,CUSTOMER_TYPE,"(-99999999999999.02, 99999999999999.02]",0.000000,0.000000,41.554330
1,CRLIMIT,"(-99999999999999.02, 397433.333]",0.846383,-0.501719,72.186085
2,CRLIMIT,"(397433.333, 750000.0]",0.269926,-0.501719,51.323334
3,CRLIMIT,"(750000.0, 1161300.0]",0.171410,-0.501719,47.757893
4,CRLIMIT,"(1161300.0, 2380000.0]",0.808742,-0.501719,70.823798


In [47]:
test = loan_c[columns]

In [48]:
test['good_bad'] = loan_c['good_bad']

/tmp/ipykernel_49181/3786374894.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['good_bad'] = loan_c['good_bad']


In [49]:
good = test[test['good_bad'] == 0]

In [50]:
bad = test[test['good_bad'] == 1]

In [51]:
def _search_score(obs, col):
  feature = [str(inter) for inter in list(WOE_dict[col]['table'].index) if obs[col].values[0] in inter][0]
  score = df_WOE[(df_WOE['Columns'] == col) & (df_WOE['Features'] == feature)]['Scores'].values[0]
  return score


In [52]:
total_scores = []
def _total_score(obs, columns = columns):
  scores = dict()
  for col in columns:
    scores[col] = _search_score(obs, col)
  total_score = sum(scores.values())

  return total_score

In [53]:
test_obs1 = test.iloc[29:30, :-1]
total_score = _total_score(test_obs1)
print(total_score)

578.3421070493217


In [57]:
cnt = {}
for i in list(df_WOE['Columns'].unique()):
  cnt[i] =1
cmt = cnt.copy()
for i in cnt:
  sub_df = df_WOE[df_WOE['Columns'] == i]
  
  a = {}
  sub_df = sub_df.reset_index()
  for j in range(len(sub_df)):

    res = sub_df['Features'][j].strip('](').split(', ')
    
    res = [float(i) for i in res]
    
    a[str(sub_df['Scores'][j])] = res
  cmt[i] = a

import json
with open("WOE_dict.json", "w") as outfile:
    json.dump(cmt, outfile)

(-99999999999999.02, 99999999999999.02]


ValueError: could not convert string to float: '('

In [ ]:
type(cnt)

dict

In [ ]:
import json
with open("WOE_dict.json", "w") as outfile:
    json.dump(cmt, outfile)

In [ ]:
test[columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15005 entries, 0 to 15004
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CUSTOMER_TYPE          15005 non-null  float64
 1   CRLIMIT                15005 non-null  float64
 2   PRINCIPAL_AMT          15005 non-null  float64
 3   PRINCIPAL_PAY_DAY      15005 non-null  int64  
 4   INTEREST_PREPAID       15005 non-null  float64
 5   NPL_DAYS               15005 non-null  int64  
 6   LAST_REPAYMENT_AMOUNT  15005 non-null  float64
 7   FEE_OUTSTANDING        15005 non-null  float64
 8   VEXPRIREDATE           15005 non-null  float64
 9   Down_payment%          15005 non-null  int64  
 10  Down payment amount    15005 non-null  float64
 11  LOAN_LENGTH            15005 non-null  int64  
dtypes: float64(8), int64(4)
memory usage: 1.4 MB


In [ ]:
test.reset_index(inplace=True)

In [ ]:
test.drop(['index'], axis = 1, inplace=True)

/tmp/ipykernel_34323/3235371512.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['index'], axis = 1, inplace=True)


In [ ]:
test.shape

(15005, 13)